In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def shrinked_cov(X):
  t, n = np.shape(X)

  S = np.cov(X, rowvar=False)  # sample cov matrix

  # Constant correlation target
  var = np.diag(S).reshape(-1, 1)
  std = np.sqrt(var)
  _var = np.tile(var, (n,))
  _std = np.tile(std, (n,))
  r_bar = (np.sum(S / (_std * _std.T)) - n) / (n * (n - 1))
  F = r_bar * (_std * _std.T)
  F[np.eye(n) == 1] = var.reshape(-1)

  # Estimate pi
  Xm = X - X.mean(axis=0)
  y = Xm ** 2
  pi_mat = np.dot(y.T, y) / t - 2 * np.dot(Xm.T, Xm) * S / t + S ** 2
  pi_hat = np.sum(pi_mat)

  # Theta matrix, expanded term by term
  term1 = np.dot((Xm ** 3).T, Xm) / t
  help_ = np.dot(Xm.T, Xm) / t
  help_diag = np.diag(help_)
  term2 = np.tile(help_diag, (n, 1)).T * S
  term3 = help_ * _var
  term4 = _var * S
  theta_mat = term1 - term2 - term3 + term4
  theta_mat[np.eye(n) == 1] = np.zeros(n)
  rho_hat = sum(np.diag(pi_mat)) + r_bar * np.sum(
      np.dot((1 / std), std.T) * theta_mat
  )

  # Estimate gamma
  gamma_hat = np.linalg.norm(S - F, "fro") ** 2

  # Compute shrinkage constant
  kappa_hat = (pi_hat - rho_hat) / gamma_hat
  delta = max(0.0, min(1.0, kappa_hat / t))aa

  # Compute shrunk covariance matrix
  shrunk_cov = delta * F + (1 - delta) * S
  return shrunk_cov

$\Sigma = \delta F + (1-\delta) S$

$f_{ii} = s \hspace{1cm}f_{ij} = \bar{r}\sqrt{s_{ii} s_{jj}} $

$L(\delta)=\Vert \delta F + (1 - \delta) S - \Sigma \Vert^2$

In [ ]:
def corr2cov(corr, std):
    cov = corr * np.outer(std, std)
    return cov   

In [ ]:
def formBlockMatrix(nBlocks, bSize, bCorr):
  block = np.ones((bSize, bSize)) * bCorr
  block[range(bSize), range(bSize)] = 1
  corr = block_diag(*([block] * nBlocks))
  return corr

In [ ]:
def formTrueMatrix(nBlocks, bSize, bCorr):
  corr0 = formBlockMatrix(nBlocks, bSize, bCorr)
  corr0 = pd.DataFrame(corr0)
  cols = corr0.columns.tolist()
  np.random.shuffle(cols)
  corr0 = corr0[cols].loc[cols].copy(deep=True)
  std0 = np.random.uniform(0.05, 0.2, corr0.shape[0])
  cov0 = corr2cov(corr0, std0)
  mu0 = np.random.normal(std0, std0, cov0.shape[0]).reshape(-1, 1)
  return mu0, cov0

In [ ]:
from scipy.linalg import block_diag
from sklearn.covariance import LedoitWolf
nBlocks = 50
bSize = 10
bCorr = .5
np.random.seed(0)
mu0, cov0 = formTrueMatrix(nBlocks, bSize, bCorr)

In [ ]:
def simCovMu(mu0, cov0, nObs, shrink=False):
  x = np.random.multivariate_normal(mu0.flatten(), cov0, size=nObs)
  mu1 = x.mean(axis=0).reshape(-1,1)
  if shrink:
    cov1 = shrinked_cov(x)
  else:
    cov1 = np.cov(x, rowvar=False)
  return mu1, cov1

In [ ]:
def optPort(cov, mu=None):
  inv = np.linalg.inv(cov)
  ones = np.ones(shape=(inv.shape[0], 1))
  if mu is None:
    mu = ones
  w = np.dot(inv, mu)
  w /= np.dot(ones.T, w)

  return w

In [ ]:
nObs = 1000
nTrials = 100

w1 = pd.DataFrame(columns=range(cov0.shape[0]), index=range(nTrials), dtype=float)
np.random.seed(0)

print("MIN VARIANCE")
minVarPortf = True
shrink = False
for i in range(nTrials):
  mu1, cov1 = simCovMu(mu0, cov0, nObs, shrink=shrink)
  if minVarPortf:
    mu1 = None
  w1.loc[i] = optPort(cov1, mu1).flatten()

w0 = optPort(cov0, None if minVarPortf else mu0)
w0 = np.repeat(w0.T, w1.shape[0], axis=0)
rmsd = np.mean((w1-w0).values.flatten()**2)**.5

print(f"NO SHRINKAGE: {rmsd}")

shrink = True
for i in range(nTrials):
  mu1, cov1 = simCovMu(mu0, cov0, nObs, shrink=shrink)
  if minVarPortf:
    mu1 = None
  w1.loc[i] = optPort(cov1, mu1).flatten()

w0 = optPort(cov0, None if minVarPortf else mu0)
w0 = np.repeat(w0.T, w1.shape[0], axis=0)
rmsd = np.mean((w1-w0).values.flatten()**2)**.5

print(f"WITH SHRINKAGE: {rmsd}")

MIN VARIANCE
NO SHRINKAGE: 0.0042147951260484515
WITH SHRINKAGE: 0.002057580131377334


In [ ]:
nObs = 1000
nTrials = 100

w1 = pd.DataFrame(columns=range(cov0.shape[0]), index=range(nTrials), dtype=float)
np.random.seed(0)

print("MAX SHARPE RATIO")
minVarPortf = False
shrink = False
for i in range(nTrials):
  mu1, cov1 = simCovMu(mu0, cov0, nObs, shrink=shrink)
  if minVarPortf:
    mu1 = None
  w1.loc[i] = optPort(cov1, mu1).flatten()

w0 = optPort(cov0, None if minVarPortf else mu0)
w0 = np.repeat(w0.T, w1.shape[0], axis=0)
rmsd = np.mean((w1-w0).values.flatten()**2)**.5

print(f"NO SHRINKAGE: {rmsd}")

shrink = True
for i in range(nTrials):
  mu1, cov1 = simCovMu(mu0, cov0, nObs, shrink=shrink)
  if minVarPortf:
    mu1 = None
  w1.loc[i] = optPort(cov1, mu1).flatten()

w0 = optPort(cov0, None if minVarPortf else mu0)
w0 = np.repeat(w0.T, w1.shape[0], axis=0)
rmsd = np.mean((w1-w0).values.flatten()**2)**.5

print(f"WITH SHRINKAGE: {rmsd}")

MAX SHARPE RATIO
NO SHRINKAGE: 0.0217839081669506
WITH SHRINKAGE: 0.009158477044664225
